In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as f
from pyspark.sql.functions import col
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

spark = SparkSession.builder.appName('ABC').getOrCreate();

In [2]:
## Lista de partidos entre los grupos desde 1986 - 2014
df_groupMatches = spark.read.option('inferSchema','true').csv('./FootballData-master/World Cups/Final Matches/world-cup-*/group_matches*',header = True);
## Lista de partidos desde octavos de final hasta la final desde 1986 -2014 mundiales
df_finalMatches = spark.read.option('inferSchema','true').csv('./FootballData-master/World Cups/Final Matches/world-cup-*/final_matches*',header=True);
## Lista de grupos desde 1986 -2014 mundiales
df_groups = spark.read.option('inferSchema','true').csv('./FootballData-master/World Cups/Final Matches/world-cup-*/groups*',header=True)
## Lista completa world matches 
df_matches2 = spark.read.option('inferSchema','true').csv('./WorldCupMatches.csv',header = True); 

In [3]:
df_groupMatches.printSchema();
df_finalMatches.printSchema();
df_groups.printSchema();
df_matches2.printSchema();

root
 |-- match_num: string (nullable = true)
 |-- round/group: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- where: string (nullable = true)
 |-- home: string (nullable = true)
 |-- home_score: string (nullable = true)
 |-- away_score: string (nullable = true)
 |-- away: string (nullable = true)
 |-- home_scorers: string (nullable = true)
 |-- away_scorers: string (nullable = true)

root
 |-- match_num: string (nullable = true)
 |-- round/group: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- where: string (nullable = true)
 |-- home: string (nullable = true)
 |-- score: string (nullable = true)
 |-- away: string (nullable = true)
 |-- home_scorers: string (nullable = true)
 |-- away_scorers: string (nullable = true)

root
 |-- Group A: string (nullable = true)
 |-- Group B: string (nullable = true)


In [9]:
df1 = df_matches2.select(f.col('Home Team Name').alias('hName'),f.col('Away Team Name').alias('aName'),f.col('Home Team Goals').alias('hGoals'),f.col('Away Team Goals').alias('aGoals'));
df1.show();

+----------+----------+------+------+
|     hName|     aName|hGoals|aGoals|
+----------+----------+------+------+
|    France|    Mexico|     4|     1|
|       USA|   Belgium|     3|     0|
|Yugoslavia|    Brazil|     2|     1|
|   Romania|      Peru|     3|     1|
| Argentina|    France|     1|     0|
|     Chile|    Mexico|     3|     0|
|Yugoslavia|   Bolivia|     4|     0|
|       USA|  Paraguay|     3|     0|
|   Uruguay|      Peru|     1|     0|
|     Chile|    France|     1|     0|
| Argentina|    Mexico|     6|     3|
|    Brazil|   Bolivia|     4|     0|
|  Paraguay|   Belgium|     1|     0|
|   Uruguay|   Romania|     4|     0|
| Argentina|     Chile|     3|     1|
| Argentina|       USA|     6|     1|
|   Uruguay|Yugoslavia|     6|     1|
|   Uruguay| Argentina|     4|     2|
|   Austria|    France|     3|     2|
|   Hungary|     Egypt|     4|     2|
+----------+----------+------+------+
only showing top 20 rows



In [74]:
df2  = df1.select('hName','hGoals','aGoals',df1.hGoals - df1.aGoals).where("hGoals > aGoals");
df2.show();
df3  = df1.select('aName','hGoals','aGoals', df1.aGoals-df1.hGoals).where("hGoals < aGoals");
df3.show();
df4 = df1.select('aName','hName','hGoals','aGoals').where("hGoals = aGoals");
df4.show();

+----------+------+------+-----------------+
|     hName|hGoals|aGoals|(hGoals - aGoals)|
+----------+------+------+-----------------+
|    France|     4|     1|                3|
|       USA|     3|     0|                3|
|Yugoslavia|     2|     1|                1|
|   Romania|     3|     1|                2|
| Argentina|     1|     0|                1|
|     Chile|     3|     0|                3|
|Yugoslavia|     4|     0|                4|
|       USA|     3|     0|                3|
|   Uruguay|     1|     0|                1|
|     Chile|     1|     0|                1|
| Argentina|     6|     3|                3|
|    Brazil|     4|     0|                4|
|  Paraguay|     1|     0|                1|
|   Uruguay|     4|     0|                4|
| Argentina|     3|     1|                2|
| Argentina|     6|     1|                5|
|   Uruguay|     6|     1|                5|
|   Uruguay|     4|     2|                2|
|   Austria|     3|     2|                1|
|   Hungar

In [73]:
df5 = df1.select(f.count('hName').alias('nRegistros'));
df5.show();
##Conteo de cuantos partidos gano cada pais como local
df6 = df2.select('hName').groupBy('hName').count().orderBy(f.col('count').desc());
df6.show();
## Conteo de cuantos partidos gano cada pais como visitante
df7 = df3.select('aName').groupBy('aName').count().orderBy(f.col('count').desc())
df7.show();

+----------+
|nRegistros|
+----------+
|       852|
+----------+

+--------------+-----+
|         hName|count|
+--------------+-----+
|        Brazil|   59|
|     Argentina|   41|
|         Italy|   38|
|    Germany FR|   29|
|       Germany|   24|
|       England|   22|
|        France|   19|
|   Netherlands|   18|
|         Spain|   16|
|       Uruguay|   16|
|        Sweden|   15|
|       Hungary|   15|
|    Yugoslavia|   13|
|  Soviet Union|   13|
|      Portugal|   12|
|        Poland|   11|
|       Belgium|   11|
|       Austria|   10|
|         Chile|    9|
|Czechoslovakia|    9|
+--------------+-----+
only showing top 20 rows

+-----------+-----+
|      aName|count|
+-----------+-----+
|      Spain|   13|
|     Brazil|   12|
|Netherlands|   11|
|    Germany|   10|
|     France|   10|
| Germany FR|    7|
|    Croatia|    7|
|      Italy|    7|
|     Mexico|    6|
|    Denmark|    5|
|    England|    4|
| Costa Rica|    4|
|Switzerland|    4|
|     Poland|    4|
|    Belgium|   

In [81]:
df_groups.show();

+------------+-------------------+--------------------+-------------+--------------+------------------+-------------+------------+
|     Group A|            Group B|             Group C|      Group D|       Group E|           Group F|      Group G|     Group H|
+------------+-------------------+--------------------+-------------+--------------+------------------+-------------+------------+
|     Germany|            England|           Argentina|     Portugal|         Italy|            Brazil|  Switzerland|       Spain|
|     Ecuador|             Sweden|         Netherlands|       Mexico|         Ghana|         Australia|       France|     Ukraine|
|      Poland|           Paraguay|         Ivory Coast|       Angola|Czech Republic|           Croatia|  South Korea|     Tunisia|
|  Costa Rica|Trinidad and Tobago|Serbia and Monten...|         Iran| United States|             Japan|         Togo|Saudi Arabia|
|South Africa|          Argentina|             England|      Germany|   Netherlands

In [90]:
df_finalMatches.select(f.col('round/group').alias('ronda'),'home').where("ronda ='Round of 16' and home != 'null'").show();

+-----------+-------------+
|      ronda|         home|
+-----------+-------------+
|Round of 16|       Brazil|
|Round of 16|     Colombia|
|Round of 16|  Netherlands|
|Round of 16|   Costa Rica|
|Round of 16|       France|
|Round of 16|      Germany|
|Round of 16|    Argentina|
|Round of 16|      Belgium|
|Round of 16|      Uruguay|
|Round of 16|United States|
|Round of 16|      Germany|
|Round of 16|    Argentina|
|Round of 16|  Netherlands|
|Round of 16|       Brazil|
|Round of 16|     Paraguay|
|Round of 16|        Spain|
|Round of 16|          0-3|
|Round of 16|          3-2|
|Round of 16|          5-4|
|Round of 16|          4-3|
+-----------+-------------+
only showing top 20 rows

